# Homework 2: Experiment tracking and model management

## By: Lorenzo Cesconetto

Check out all the questions [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2025/02-experiment-tracking/homework.md).


# Question 1: Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


In [1]:
!mlflow --version

mlflow, version 2.22.0


# Questions 2: Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip.

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2023 data),
- save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

- 1
- 3
- 4
- 7


In [2]:
from urllib.request import urlretrieve
import os

for month in range(1, 4):
    month_str = str(month).zfill(2)
    url = (
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-{month_str}.parquet"
    )
    filename = f"./tmp_data/{os.path.basename(url)}"
    urlretrieve(url, filename)

In [3]:
!python preprocess_data.py --raw_data_path tmp_data --dest_path ./output

In [5]:
!echo "File count $(ls output | wc -l)" 

File count        4


# Question 3: Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework).

The script will:

- load the datasets produced by the previous step,
- train the model on the training set,
- calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

- 2
- 4
- 8
- 10


In [6]:
!python train.py

2025/05/25 21:16:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3042f2e126264cd4b2db4e1420dc3a87', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/05/25 21:16:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


In [7]:
!echo "min_samples_split: $(cat $(ls ./mlruns/0/*/params/min_samples_split | head -1))"


min_samples_split: 2


## Question 4: Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

- launch the tracking server on your local machine,
- select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

- `default-artifact-root`
- `serve-artifacts`
- `artifacts-only`
- `artifacts-destination`


---

The answer is `default-artifact-root`, the instructor came across that exact same error during the classes.
But in my case it worked without it, it's probably because I am using a later version of `mlflow`!


In [7]:
!mlflow server --backend-store-uri sqlite:///backend.db

2025/05/25 21:13:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/25 21:13:53 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [8]:
!mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts

[2025-05-25 21:14:00 -0300] [20472] [INFO] Starting gunicorn 23.0.0
[2025-05-25 21:14:00 -0300] [20472] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-05-25 21:14:00 -0300] [20472] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 48] Address already in use
[2025-05-25 21:14:01 -0300] [20472] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-05-25 21:14:01 -0300] [20472] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 48] Address already in use
[2025-05-25 21:14:02 -0300] [20472] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-05-25 21:14:02 -0300] [20472] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 48] Address already in use
[2025-05-25 21:14:03 -0300] [20472] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-05-25 21:14:03 -0300] [20472] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 48] Address already in use
[2025-05-25 21:14:04 -0300] [20472] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-05-25 21:14:04 -0300] [20472] [

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`.
We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

- the list of hyperparameters that are passed to the `objective` function during the optimization,
- the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

- 4.817
- 5.335
- 5.818
- 6.336


In [8]:
!python hpo.py

2025/05/25 21:17:21 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run debonair-wasp-98 at: http://127.0.0.1:5000/#/experiments/1/runs/2439abf7a2ca43d3a620927223ae2446

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run enchanting-moth-178 at: http://127.0.0.1:5000/#/experiments/1/runs/02239dca8bb14ecd8012d1e495af47ba

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run bittersweet-colt-855 at: http://127.0.0.1:5000/#/experiments/1/runs/bb95264eabbc4da7bb45761d61dc7a18

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run fun-fox-101 at: http://127.0.0.1:5000/#/experiments/1/runs/a37ad49a20cf463ab4a63d6ff567d9a9

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1                    

🏃 View run calm-lark-451 at: http://127.0.0.1:5000/#/experiments/1/runs/683005979d764b83bced7075928f

---

**Answer:** best RMSE = 5.335


# Question 6: Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production.
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs.
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

- 5.060
- 5.567
- 6.061
- 6.568


In [9]:
!python register_model.py

2025/05/25 21:17:55 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2025/05/25 21:18:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run youthful-croc-518 at: http://127.0.0.1:5000/#/experiments/2/runs/251a87a03e70411492337fcdf4c749f2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
2025/05/25 21:18:10 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run unique-hen-238 at: http://127.0.0.1:5000/#/experiments/2/runs/79ef432446b4474281a92dbd59a6b31a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
2025/05/25 21:18:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version spec

---

**Answer:** best test RMSE = 5.567
